In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

In [15]:
from sklearn import set_config

set_config(display='diagram')

In [2]:
df = pd.read_csv('datasets/train.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'], inplace=True)

In [5]:
df.shape

(891, 8)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [7]:
df.isnull().mean()*100

Survived     0.000000
Pclass       0.000000
Sex          0.000000
Age         19.865320
SibSp        0.000000
Parch        0.000000
Fare         0.000000
Embarked     0.224467
dtype: float64

In [8]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [9]:
X = df.drop(columns=['Survived'])

y = df['Survived']

In [10]:
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)

In [12]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
410,3,male,NaN,0,0,7.8958,S
638,3,female,41.0,0,5,39.6875,S
395,3,male,22.0,0,0,7.7958,S
221,2,male,27.0,0,0,13.0000,S
800,2,male,34.0,0,0,13.0000,S


## Let's make pipeline

In [13]:
numerical_col = ['Age', 'Fare']

numerical_pipeline = Pipeline(steps=[
    ('imputeMissingValue', SimpleImputer(strategy="median")),
    ('scaling', StandardScaler())
])


categorical_col = ['Embarked', 'Sex']

categorical_pipeline = Pipeline(steps=[
    ('imputeMissingValue2', SimpleImputer(strategy="most_frequent")),
    ('encoding', OneHotEncoder(handle_unknown="ignore"))
])

In [14]:
col_transformers = ColumnTransformer(transformers=[
    ('numerical', numerical_pipeline, numerical_col),
    ('categorical', categorical_pipeline, categorical_col)
])

In [16]:
col_transformers

ColumnTransformer(transformers=[('numerical',
                                 Pipeline(steps=[('imputeMissingValue',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaling',
                                                  StandardScaler())]),
                                 ['Age', 'Fare']),
                                ('categorical',
                                 Pipeline(steps=[('imputeMissingValue2',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoding',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Embarked', 'Sex'])])

In [17]:
model = Pipeline(steps=[
    ('transformers', col_transformers),
    ('classifier', LogisticRegression())
])

In [18]:
model

Pipeline(steps=[('transformers',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputeMissingValue',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaling',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputeMissingValue2',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoding',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Embarked', 'Sex'])])),
                ('classifier', LogisticRegression())])

In [ ]:
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'preprocessor__cat__imputer__strategy': ['most_frequent', 'constant'],
    'classifier__C': [0.1, 1.0, 10, 100]
}

In [19]:
param_grid = {
    'transformers__numerical__imputeMissingValue__strategy' : ['mean', 'median'],
    'transformers__categorical__imputeMissingValue2__strategy' : ['most_frequent', 'constant'],
    'classifier__C' : [0.1, 1.0, 10, 100]
}

In [20]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=15)

In [21]:
grid_search.fit(X=X_train, y=y_train)

GridSearchCV(cv=15,
             estimator=Pipeline(steps=[('transformers',
                                        ColumnTransformer(transformers=[('numerical',
                                                                         Pipeline(steps=[('imputeMissingValue',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaling',
                                                                                          StandardScaler())]),
                                                                         ['Age',
                                                                          'Fare']),
                                                                        ('categorical',
                                                                         Pipeline(steps=[('imputeMissingValue2',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('encoding',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['Embarked',
                                                                          'Sex'])])),
                                       ('classifier', LogisticRegression())]),
             param_grid={'classifier__C': [0.1, 1.0, 10, 100],
                         'transformers__categorical__imputeMissingValue2__strategy': ['most_frequent',
                                                                                      'constant'],
                         'transformers__numerical__imputeMissingValue__strategy': ['mean',
                                                                                   'median']})

In [22]:
grid_search.best_params_

{'classifier__C': 1.0,
 'transformers__categorical__imputeMissingValue2__strategy': 'most_frequent',
 'transformers__numerical__imputeMissingValue__strategy': 'mean'}

In [23]:
print(f"Internal CV score: {grid_search.best_score_:.3f}")

Internal CV score: 0.783


In [24]:
cv_results = pd.DataFrame(data=grid_search.cv_results_)

In [25]:
cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__C,param_transformers__categorical__imputeMissingValue2__strategy,param_transformers__numerical__imputeMissingValue__strategy,params,split0_test_score,split1_test_score,...,split8_test_score,split9_test_score,split10_test_score,split11_test_score,split12_test_score,split13_test_score,split14_test_score,mean_test_score,std_test_score,rank_test_score
0,0.024648,0.012447,0.007594,0.001780,0.1,most_frequent,mean,"{'classifier__C': 0.1, 'transformers__categori...",0.729167,0.708333,...,0.829787,0.829787,0.829787,0.808511,0.765957,0.765957,0.744681,0.781176,0.050180,14
1,0.018386,0.000951,0.006462,0.000618,0.1,most_frequent,median,"{'classifier__C': 0.1, 'transformers__categori...",0.729167,0.708333,...,0.829787,0.829787,0.829787,0.808511,0.765957,0.765957,0.744681,0.781176,0.050180,14
2,0.023982,0.007867,0.008727,0.003393,0.1,constant,mean,"{'classifier__C': 0.1, 'transformers__categori...",0.750000,0.708333,...,0.829787,0.829787,0.829787,0.808511,0.765957,0.765957,0.744681,0.782565,0.048995,13
3,0.021118,0.002245,0.007394,0.000879,0.1,constant,median,"{'classifier__C': 0.1, 'transformers__categori...",0.729167,0.708333,...,0.829787,0.829787,0.829787,0.808511,0.765957,0.765957,0.744681,0.781176,0.050180,14
4,0.018120,0.000805,0.006262,0.000771,1.0,most_frequent,mean,"{'classifier__C': 1.0, 'transformers__categori...",0.729167,0.708333,...,0.829787,0.829787,0.829787,0.808511,0.765957,0.765957,0.744681,0.782595,0.052960,1
5,0.034508,0.026680,0.011724,0.011809,1.0,most_frequent,median,"{'classifier__C': 1.0, 'transformers__categori...",0.729167,0.708333,...,0.829787,0.829787,0.829787,0.808511,0.765957,0.765957,0.744681,0.782595,0.052960,1
6,0.019386,0.001624,0.006528,0.000718,1.0,constant,mean,"{'classifier__C': 1.0, 'transformers__categori...",0.729167,0.708333,...,0.829787,0.829787,0.829787,0.808511,0.765957,0.765957,0.744681,0.782595,0.052960,1
7,0.019052,0.001768,0.006328,0.000698,1.0,constant,median,"{'classifier__C': 1.0, 'transformers__categori...",0.729167,0.708333,...,0.829787,0.829787,0.829787,0.808511,0.765957,0.765957,0.744681,0.782595,0.052960,1
8,0.017720,0.002111,0.006662,0.000788,10,most_frequent,mean,"{'classifier__C': 10, 'transformers__categoric...",0.729167,0.708333,...,0.829787,0.829787,0.829787,0.808511,0.765957,0.765957,0.744681,0.782595,0.052960,1
9,0.017587,0.001305,0.006195,0.000541,10,most_frequent,median,"{'classifier__C': 10, 'transformers__categoric...",0.729167,0.708333,...,0.829787,0.829787,0.829787,0.808511,0.765957,0.765957,0.744681,0.782595,0.052960,1


In [26]:
cv_results.columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_classifier__C',
       'param_transformers__categorical__imputeMissingValue2__strategy',
       'param_transformers__numerical__imputeMissingValue__strategy', 'params',
       'split0_test_score', 'split1_test_score', 'split2_test_score',
       'split3_test_score', 'split4_test_score', 'split5_test_score',
       'split6_test_score', 'split7_test_score', 'split8_test_score',
       'split9_test_score', 'split10_test_score', 'split11_test_score',
       'split12_test_score', 'split13_test_score', 'split14_test_score',
       'mean_test_score', 'std_test_score', 'rank_test_score'],
      dtype='object')

In [27]:
cv_results = cv_results.sort_values(by="mean_test_score", ascending=False)

In [28]:
col = ['param_classifier__C',
       'param_transformers__categorical__imputeMissingValue2__strategy',
       'param_transformers__numerical__imputeMissingValue__strategy',
       'mean_test_score']


cv_results[col]

,param_classifier__C,param_transformers__categorical__imputeMissingValue2__strategy,param_transformers__numerical__imputeMissingValue__strategy,mean_test_score
4,1.0,most_frequent,mean,0.782595
5,1.0,most_frequent,median,0.782595
6,1.0,constant,mean,0.782595
7,1.0,constant,median,0.782595
8,10,most_frequent,mean,0.782595
9,10,most_frequent,median,0.782595
10,10,constant,mean,0.782595
11,10,constant,median,0.782595
12,100,most_frequent,mean,0.782595
13,100,most_frequent,median,0.782595
